In [9]:
import pandas as pd
import glob
from fuzzywuzzy import fuzz

Data source: https://data.cityofnewyork.us/Education/2019-DOE-High-School-Directory/uq7m-95z8/about_data \
We used all the high school DOE datasets from 2016 to 2021 \
Street code data source: https://data.cityofnewyork.us/City-Government/Street-Name-Dictionary/w4v2-rv6b

In [10]:
# Define column widths
colspecs = [(2, 34), (36, 37), (37, 42)]

# Read the file
sc = pd.read_fwf('../datasets/street_dict.txt', colspecs=colspecs, header=None, names=["address_name", "borough", "street_code"], dtype='string')

# Mapping based on documentation from here: https://data.cityofnewyork.us/City-Government/Street-Name-Dictionary/w4v2-rv6b/about_data
sc.loc[sc.borough == "1", "borough"] = 'NY'
sc.loc[sc.borough == "2", "borough"] = 'BX'
sc.loc[sc.borough == "3", "borough"] = 'K'
sc.loc[sc.borough == "4", "borough"] = 'Q'
sc.loc[sc.borough == "5", "borough"] = 'R'

Fuzzy match based on address and exact match on borough. After combine

In [11]:
base_path = '../datasets/hs/'
years = ['2021', '2016', '2017', '2018', '2019', '2020']
files = [file for year in years for file in glob.glob(base_path + year + ".csv")]

def fuzzy_match(row, threshold=97):  # increase the threshold for a near-perfect match
    return fuzz.token_sort_ratio(row['primary_address_line_1'], row['address_name']) > threshold

for file in files:
    hs = pd.read_csv(file) 

    cols_of_use = ['boro', 'school_name', 'primary_address_line_1', 'bus', 'subway', 'total_students', 'city', 'DataYear']
    hs = hs[cols_of_use]


    hs.loc[hs.boro == "M", "boro"] = 'NY'
    hs.loc[hs.boro == "X", "boro"] = 'BX'

    hs.loc[hs.boro == "Manhattan", "boro"] = 'NY'
    hs.loc[hs.boro == "Bronx", "boro"] = 'BX'
    hs.loc[hs.boro == "Queens", "boro"] = 'Q'
    hs.loc[hs.boro == "Staten Island", "boro"] = 'R'
    hs.loc[hs.boro == "Brooklyn", "boro"] = 'K'

    # Convert the columns to lowercase to make the matching easier
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.lower()
    sc['address_name'] = sc['address_name'].str.lower()

    # Remove extra whitespaces between words in 'primary_address_line_1' and 'address_name'
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('\s+', ' ', regex=True)
    sc['address_name'] = sc['address_name'].str.replace('\s+', ' ', regex=True)

    # Remove suffixes like 'th', 'st', 'nd', 'rd' from 'primary_address_line_1'
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('(\d+)(st|nd|rd|th)', r'\1', regex=True)
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('st.', 'st')
    hs['primary_address_line_1'] = hs['primary_address_line_1'].str.replace('ave.', 'avenue')
    
    # First, join on borough column
    merged = sc.merge(hs, left_on='borough', right_on='boro', suffixes=('', '_y'), how='inner')
    merged = merged.drop(columns='boro')

    merged['primary_address_line_1'] = merged['primary_address_line_1'].str.replace('^\S+\s*', '', regex=True)

    # Fuzzy match entry addreses. Not needed anymore. I used it to see what are the difference in the street so I can
    # add the rules above
    mask = merged.apply(fuzzy_match, axis=1)

    temp = merged[mask]
    diff = temp[temp.address_name != temp.primary_address_line_1]
    print(f"Year: {file} \n New: {len(temp)} \n Old: {len(hs)}")
    print("------------------------")
    merged = merged.drop(columns='primary_address_line_1')

    if '2021' in file:
        merged[mask].to_csv('../datasets/hs/combined_hs_with_street_codes.csv', mode='w', header=True, index=False)

    merged[mask].to_csv('../datasets/hs/combined_hs_with_street_codes.csv', mode='a', header=False, index=False)

Year: ../datasets/hs/2021.csv 
 New: 421 
 Old: 442
------------------------
Year: ../datasets/hs/2016.csv 
 New: 412 
 Old: 437
------------------------
Year: ../datasets/hs/2017.csv 
 New: 416 
 Old: 440
------------------------
Year: ../datasets/hs/2018.csv 
 New: 414 
 Old: 435
------------------------
Year: ../datasets/hs/2019.csv 
 New: 408 
 Old: 427
------------------------
Year: ../datasets/hs/2020.csv 
 New: 408 
 Old: 427
------------------------


Make the file a parquet since append is not supported for parquet file types

In [14]:
hssc = pd.read_csv("../datasets/hs/combined_hs_with_street_codes.csv", low_memory=False)
hssc.to_parquet("../datasets/hs/combined_hs_with_street_codes.parquet", compression='snappy')

In [15]:
hspk = pd.read_parquet("../datasets/hs/combined_hs_with_street_codes.parquet")
hspk = hspk.groupby(['street_code', 'borough', 'address_name', 'DataYear']).size().reset_index(name='school_count')
hspk['street_code'] = hspk['street_code'].astype('string')


hspk = sc.merge(hspk, left_on=['street_code', 'borough'], right_on=['street_code','borough'], suffixes=('', '_y'), how="left")
hspk.drop(columns=['address_name_y'], inplace=True)
hspk.fillna({'school_count': 0}, inplace=True)
hspk['school_count'] = hspk['school_count'].astype(int)


hspk.to_parquet("../datasets/hs/hs_per_street.parquet", compression='snappy')

In [17]:
t = pd.read_parquet("../datasets/hs/combined_hs_with_street_codes.parquet")
t.head(1000)

,address_name,borough,street_code,school_name,bus,subway,total_students,city,DataYear
0,7 avenue,NY,10610,Frederick Douglass Academy,"Bx13, Bx19, Bx6, M1, M10, M102, M2, M3, M7","3 to Harlem-148th St; B, D to 155th St",1072.0,Manhattan,2021
1,10 avenue,NY,11010,Independence High School,"BxM2, M10, M104, M11, M12, M20, M31, M5, M50, ...","1, A, C, B, D to 59th St-Columbus Circle; E to...",342.0,Manhattan,2021
2,academy street,NY,11310,Inwood Early College for Health and Informatio...,"Bx12, Bx12-SBS, Bx20, Bx7, BxM1, M100, M3, M4","1, A to Dyckman St",459.0,Manhattan,2021
3,academy street,NY,11310,High School for Excellence and Innovation,"Bx12, Bx12-SBS, Bx20, Bx7, BxM1, M100, M3, M4","1, A to Dyckman St",189.0,Manhattan,2021
4,amsterdam avenue,NY,11710,Maxine Greene High School for Imaginative Inqu...,"BxM2, M10, M104, M11, M12, M20, M31, M5, M57, ...","1 to 66th St-Lincoln Center; 2, 3 to 72nd St; ...",285.0,Manhattan,2021
...,...,...,...,...,...,...,...,...,...
995,east gun hill road,BX,29320,Bronx Lab School,"Bx28, Bx30, Bx39, Bx41, Bx8, BxM11","2, 5 to Gun Hill Rd",471.0,BRONX,2016
996,east gun hill road,BX,29320,High School of Computers and Technology,"Bx28, Bx30, Bx39, Bx41, Bx8, BxM11","2, 5 to Gun Hill Rd",567.0,BRONX,2016
997,east gun hill road,BX,29320,Bronx Academy of Health Careers,"Bx28, Bx30, Bx39, Bx41, Bx8, BxM11","2, 5 to Gun Hill Rd",465.0,BRONX,2016
998,east gun hill road,BX,29320,High School for Contemporary Arts,"Bx28, Bx30, Bx39, Bx41, Bx8, BxM11","2, 5 to Gun Hill Rd",493.0,BRONX,2016
